In [36]:
import pandas as pd
import numpy as np
import re

import json
import geopandas as gpd

In [37]:
rawdata_path = 'rawdata/'
data_path = 'data/'

In [38]:
### 통계청 행정구역 분류

df_code = pd.read_excel(f'{rawdata_path}/region_code/한국행정구역분류_2023.4.1.기준.xlsx', sheet_name='2-2. 연계표_행정동 및 법정동(기준시점)', header=1)
df_code = df_code[df_code['시도'] == '서울특별시']
df_code

c:\Users\tjoeun\miniconda3\envs\myproject1\lib\site-packages\openpyxl\worksheet\header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")


,시도,시군구,행정구역명,행정동\n(행정기관명),법정동,행정구역분류,행정기관코드,행정기관 생성일,법정동코드,법정동 관할구역\n분할여부,행정동 영문명칭,비고
0,서울특별시,서울특별시,서울특별시,서울특별시,서울특별시,11.0,1100000000,19880423,1100000000,NaN,Seoul,NaN
1,서울특별시,종로구,종로구,종로구,종로구,11010.0,1111000000,19880423,1111000000,NaN,Jongno-gu,NaN
2,서울특별시,종로구,청운효자동,청운효자동,청운동,11010720.0,1111051500,20081101,1111010100,NaN,Cheongunhyoja-dong,NaN
3,서울특별시,종로구,청운효자동,청운효자동,신교동,11010720.0,1111051500,20081101,1111010200,NaN,Cheongunhyoja-dong,NaN
4,서울특별시,종로구,청운효자동,청운효자동,궁정동,11010720.0,1111051500,20081101,1111010300,NaN,Cheongunhyoja-dong,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
762,서울특별시,강동구,성내2동,성내제2동,성내동,11250660.0,1174065000,19880423,1174010800,분할연계,Seongnae 2(i)-dong,NaN
763,서울특별시,강동구,성내3동,성내제3동,성내동,11250670.0,1174066000,19880423,1174010800,분할연계,Seongnae 3(sam)-dong,NaN
764,서울특별시,강동구,길동,길동,길동,11250740.0,1174068500,20080707,1174010500,NaN,Gil-dong,NaN
765,서울특별시,강동구,둔촌1동,둔촌제1동,둔촌동,11250700.0,1174069000,19880423,1174010600,분할연계,Dunchon 1(il)-dong,NaN


In [39]:
df_code = df_code[df_code['행정구역분류'] > 100000]
df_code.drop(columns=['시도', '행정동\n(행정기관명)', '행정기관 생성일', '법정동 관할구역\n분할여부', '행정동 영문명칭', '비고'], inplace= True)
df_code.reset_index(drop = True, inplace= True)
df_code.head()

C:\Users\tjoeun\AppData\Local\Temp\ipykernel_6852\3455889288.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_code.drop(columns=['시도', '행정동\n(행정기관명)', '행정기관 생성일', '법정동 관할구역\n분할여부', '행정동 영문명칭', '비고'], inplace= True)


,시군구,행정구역명,법정동,행정구역분류,행정기관코드,법정동코드
0,종로구,청운효자동,청운동,11010720.0,1111051500,1111010100
1,종로구,청운효자동,신교동,11010720.0,1111051500,1111010200
2,종로구,청운효자동,궁정동,11010720.0,1111051500,1111010300
3,종로구,청운효자동,효자동,11010720.0,1111051500,1111010400
4,종로구,청운효자동,창성동,11010720.0,1111051500,1111010500


In [40]:
df_code.columns = ['시군구', '행정동', '법정동', 'ADM_CD', '행정동코드', '법정동코드']
df_code.head()

,시군구,행정동,법정동,ADM_CD,행정동코드,법정동코드
0,종로구,청운효자동,청운동,11010720.0,1111051500,1111010100
1,종로구,청운효자동,신교동,11010720.0,1111051500,1111010200
2,종로구,청운효자동,궁정동,11010720.0,1111051500,1111010300
3,종로구,청운효자동,효자동,11010720.0,1111051500,1111010400
4,종로구,청운효자동,창성동,11010720.0,1111051500,1111010500


In [41]:
# 행안부 통계코드 분류

hang = pd.read_excel(f'{rawdata_path}/region_code/행안부_동코드 정리.xlsx')
hang.head()

,행정동코드,시도명,시군구명,읍면동명,법정동코드,동리명,생성일자,말소일자
0,1100000000,서울특별시,NaN,NaN,1100000000,서울특별시,19880423,NaN
1,1111000000,서울특별시,종로구,NaN,1111000000,종로구,19880423,NaN
2,1111051500,서울특별시,종로구,청운효자동,1111010100,청운동,20081101,NaN
3,1111051500,서울특별시,종로구,청운효자동,1111010200,신교동,20081101,NaN
4,1111051500,서울특별시,종로구,청운효자동,1111010300,궁정동,20081101,NaN


In [42]:
hang = hang[hang['시도명'] == '서울특별시']
hang = hang[hang['읍면동명'].notna()]
hang.head(8)

,행정동코드,시도명,시군구명,읍면동명,법정동코드,동리명,생성일자,말소일자
2,1111051500,서울특별시,종로구,청운효자동,1111010100,청운동,20081101,NaN
3,1111051500,서울특별시,종로구,청운효자동,1111010200,신교동,20081101,NaN
4,1111051500,서울특별시,종로구,청운효자동,1111010300,궁정동,20081101,NaN
5,1111051500,서울특별시,종로구,청운효자동,1111010400,효자동,20081101,NaN
6,1111051500,서울특별시,종로구,청운효자동,1111010500,창성동,20081101,NaN
7,1111051500,서울특별시,종로구,청운효자동,1111010800,통인동,20081101,NaN
8,1111051500,서울특별시,종로구,청운효자동,1111010900,누상동,20081101,NaN
9,1111051500,서울특별시,종로구,청운효자동,1111011000,누하동,20081101,NaN


In [43]:
hang.drop(columns=['시도명', '생성일자', '말소일자'], inplace = True)
hang.columns = ['행정동코드', '시군구', '행정동', '법정동코드', '법정동']
hang.reset_index(drop=True, inplace=True)
hang

,행정동코드,시군구,행정동,법정동코드,법정동
0,1111051500,종로구,청운효자동,1111010100,청운동
1,1111051500,종로구,청운효자동,1111010200,신교동
2,1111051500,종로구,청운효자동,1111010300,궁정동
3,1111051500,종로구,청운효자동,1111010400,효자동
4,1111051500,종로구,청운효자동,1111010500,창성동
...,...,...,...,...,...
740,1174065000,강동구,성내제2동,1174010800,성내동
741,1174066000,강동구,성내제3동,1174010800,성내동
742,1174068500,강동구,길동,1174010500,길동
743,1174069000,강동구,둔촌제1동,1174010600,둔촌동


In [44]:
df_code

,시군구,행정동,법정동,ADM_CD,행정동코드,법정동코드
0,종로구,청운효자동,청운동,11010720.0,1111051500,1111010100
1,종로구,청운효자동,신교동,11010720.0,1111051500,1111010200
2,종로구,청운효자동,궁정동,11010720.0,1111051500,1111010300
3,종로구,청운효자동,효자동,11010720.0,1111051500,1111010400
4,종로구,청운효자동,창성동,11010720.0,1111051500,1111010500
...,...,...,...,...,...,...
736,강동구,성내2동,성내동,11250660.0,1174065000,1174010800
737,강동구,성내3동,성내동,11250670.0,1174066000,1174010800
738,강동구,길동,길동,11250740.0,1174068500,1174010500
739,강동구,둔촌1동,둔촌동,11250700.0,1174069000,1174010600


In [45]:
hang[hang['행정동'] == '청운효자동']

,행정동코드,시군구,행정동,법정동코드,법정동
0,1111051500,종로구,청운효자동,1111010100,청운동
1,1111051500,종로구,청운효자동,1111010200,신교동
2,1111051500,종로구,청운효자동,1111010300,궁정동
3,1111051500,종로구,청운효자동,1111010400,효자동
4,1111051500,종로구,청운효자동,1111010500,창성동
5,1111051500,종로구,청운효자동,1111010800,통인동
6,1111051500,종로구,청운효자동,1111010900,누상동
7,1111051500,종로구,청운효자동,1111011000,누하동
8,1111051500,종로구,청운효자동,1111011100,옥인동
9,1111051500,종로구,청운효자동,1111011900,세종로


In [46]:
df_code[df_code['행정동'] == '청운효자동']

,시군구,행정동,법정동,ADM_CD,행정동코드,법정동코드
0,종로구,청운효자동,청운동,11010720.0,1111051500,1111010100
1,종로구,청운효자동,신교동,11010720.0,1111051500,1111010200
2,종로구,청운효자동,궁정동,11010720.0,1111051500,1111010300
3,종로구,청운효자동,효자동,11010720.0,1111051500,1111010400
4,종로구,청운효자동,창성동,11010720.0,1111051500,1111010500
5,종로구,청운효자동,통인동,11010720.0,1111051500,1111010800
6,종로구,청운효자동,누상동,11010720.0,1111051500,1111010900
7,종로구,청운효자동,누하동,11010720.0,1111051500,1111011000
8,종로구,청운효자동,옥인동,11010720.0,1111051500,1111011100
9,종로구,청운효자동,세종로,11010720.0,1111051500,1111011900


In [47]:
temp = df_code.merge(hang,
                     how = 'outer',
                     on= ['시군구', '행정동코드', '법정동코드'],
                     suffixes=['1', '2'])

In [48]:
temp

,시군구,행정동1,법정동1,ADM_CD,행정동코드,법정동코드,행정동2,법정동2
0,강남구,신사동,신사동,11230510.0,1168051000,1168010700,신사동,신사동
1,강남구,신사동,압구정동,11230510.0,1168051000,1168011000,신사동,압구정동
2,강남구,논현1동,논현동,11230520.0,1168052100,1168010800,논현1동,논현동
3,강남구,논현2동,논현동,11230530.0,1168053100,1168010800,논현2동,논현동
4,강남구,압구정동,신사동,11230770.0,1168054500,1168010700,압구정동,신사동
...,...,...,...,...,...,...,...,...
740,중랑구,묵2동,묵동,11070650.0,1126063000,1126010400,묵제2동,묵동
741,중랑구,망우본동,망우동,11070730.0,1126065500,1126010500,망우본동,망우동
742,중랑구,망우3동,망우동,11070680.0,1126066000,1126010500,망우제3동,망우동
743,중랑구,신내1동,신내동,11070690.0,1126068000,1126010600,신내1동,신내동


In [49]:
temp[temp.isna().any(axis=1)]

,시군구,행정동1,법정동1,ADM_CD,행정동코드,법정동코드,행정동2,법정동2
76,강서구,NaN,NaN,NaN,1150060300,1150010200,가양제1동,등촌동
97,강서구,NaN,NaN,NaN,1150064100,1150010500,방화제3동,마곡동
172,노원구,NaN,NaN,NaN,1135062400,1135010500,중계4동,상계동
277,서대문구,NaN,NaN,NaN,1141055500,1141011200,북아현동,대현동


In [50]:
temp.dropna(inplace=True)
region_code = temp.copy()

In [51]:
region_code['행정동1'] = region_code['행정동1'].apply(lambda x: re.sub(r'\.', '', x))
region_code['행정동1'] = region_code['행정동1'].apply(lambda x: re.sub(r'\·', '', x))

region_code['행정동2'] = region_code['행정동2'].apply(lambda x: re.sub(r'\.', '', x))
region_code['행정동2'] = region_code['행정동2'].apply(lambda x: re.sub(r'\·', '', x))

region_code['법정동1'] = region_code['법정동1'].apply(lambda x: re.sub(r'\.', '', x))
region_code['법정동1'] = region_code['법정동1'].apply(lambda x: re.sub(r'\·', '', x))

region_code['법정동2'] = region_code['법정동2'].apply(lambda x: re.sub(r'\.', '', x))
region_code['법정동2'] = region_code['법정동2'].apply(lambda x: re.sub(r'\·', '', x))




In [52]:
region_code = region_code[['ADM_CD', '행정동코드', '법정동코드', '시군구', '행정동1', '행정동2', '법정동1', '법정동2']]
region_code['key1'] = region_code['시군구'] + ' ' + region_code['행정동1']
region_code['key2'] = region_code['시군구'] + ' ' + region_code['행정동2']
region_code

,ADM_CD,행정동코드,법정동코드,시군구,행정동1,행정동2,법정동1,법정동2,key1,key2
0,11230510.0,1168051000,1168010700,강남구,신사동,신사동,신사동,신사동,강남구 신사동,강남구 신사동
1,11230510.0,1168051000,1168011000,강남구,신사동,신사동,압구정동,압구정동,강남구 신사동,강남구 신사동
2,11230520.0,1168052100,1168010800,강남구,논현1동,논현1동,논현동,논현동,강남구 논현1동,강남구 논현1동
3,11230530.0,1168053100,1168010800,강남구,논현2동,논현2동,논현동,논현동,강남구 논현2동,강남구 논현2동
4,11230770.0,1168054500,1168010700,강남구,압구정동,압구정동,신사동,신사동,강남구 압구정동,강남구 압구정동
...,...,...,...,...,...,...,...,...,...,...
740,11070650.0,1126063000,1126010400,중랑구,묵2동,묵제2동,묵동,묵동,중랑구 묵2동,중랑구 묵제2동
741,11070730.0,1126065500,1126010500,중랑구,망우본동,망우본동,망우동,망우동,중랑구 망우본동,중랑구 망우본동
742,11070680.0,1126066000,1126010500,중랑구,망우3동,망우제3동,망우동,망우동,중랑구 망우3동,중랑구 망우제3동
743,11070690.0,1126068000,1126010600,중랑구,신내1동,신내1동,신내동,신내동,중랑구 신내1동,중랑구 신내1동


In [53]:
region_code[region_code['행정동1'] == '청운효자동']

,ADM_CD,행정동코드,법정동코드,시군구,행정동1,행정동2,법정동1,법정동2,key1,key2
544,11010720.0,1111051500,1111010100,종로구,청운효자동,청운효자동,청운동,청운동,종로구 청운효자동,종로구 청운효자동
545,11010720.0,1111051500,1111010200,종로구,청운효자동,청운효자동,신교동,신교동,종로구 청운효자동,종로구 청운효자동
546,11010720.0,1111051500,1111010300,종로구,청운효자동,청운효자동,궁정동,궁정동,종로구 청운효자동,종로구 청운효자동
547,11010720.0,1111051500,1111010400,종로구,청운효자동,청운효자동,효자동,효자동,종로구 청운효자동,종로구 청운효자동
548,11010720.0,1111051500,1111010500,종로구,청운효자동,청운효자동,창성동,창성동,종로구 청운효자동,종로구 청운효자동
549,11010720.0,1111051500,1111010800,종로구,청운효자동,청운효자동,통인동,통인동,종로구 청운효자동,종로구 청운효자동
550,11010720.0,1111051500,1111010900,종로구,청운효자동,청운효자동,누상동,누상동,종로구 청운효자동,종로구 청운효자동
551,11010720.0,1111051500,1111011000,종로구,청운효자동,청운효자동,누하동,누하동,종로구 청운효자동,종로구 청운효자동
552,11010720.0,1111051500,1111011100,종로구,청운효자동,청운효자동,옥인동,옥인동,종로구 청운효자동,종로구 청운효자동
553,11010720.0,1111051500,1111011900,종로구,청운효자동,청운효자동,세종로,세종로,종로구 청운효자동,종로구 청운효자동


In [54]:
region_code[region_code['법정동1'] != region_code['법정동2']]

,ADM_CD,행정동코드,법정동코드,시군구,행정동1,행정동2,법정동1,법정동2,key1,key2


In [55]:
region_code[region_code['행정동1'] != region_code['행정동2']]

,ADM_CD,행정동코드,법정동코드,시군구,행정동1,행정동2,법정동1,법정동2,key1,key2
32,11250760.0,1174052500,1174010300,강동구,상일1동,상일제1동,상일동,상일동,강동구 상일1동,강동구 상일제1동
33,11250770.0,1174052600,1174010300,강동구,상일2동,상일제2동,상일동,상일동,강동구 상일2동,강동구 상일제2동
34,11250530.0,1174053000,1174010100,강동구,명일1동,명일제1동,명일동,명일동,강동구 명일1동,강동구 명일제1동
35,11250540.0,1174054000,1174010100,강동구,명일2동,명일제2동,명일동,명일동,강동구 명일2동,강동구 명일제2동
36,11250550.0,1174055000,1174010200,강동구,고덕1동,고덕제1동,고덕동,고덕동,강동구 고덕1동,강동구 고덕제1동
...,...,...,...,...,...,...,...,...,...,...
737,11070610.0,1126060000,1126010300,중랑구,중화1동,중화제1동,중화동,중화동,중랑구 중화1동,중랑구 중화제1동
738,11070620.0,1126061000,1126010300,중랑구,중화2동,중화제2동,중화동,중화동,중랑구 중화2동,중랑구 중화제2동
739,11070640.0,1126062000,1126010400,중랑구,묵1동,묵제1동,묵동,묵동,중랑구 묵1동,중랑구 묵제1동
740,11070650.0,1126063000,1126010400,중랑구,묵2동,묵제2동,묵동,묵동,중랑구 묵2동,중랑구 묵제2동


In [56]:
region_code.to_csv(f'{data_path}/region_code.csv', index=False)

In [57]:
dong_shp = gpd.read_file(f'{rawdata_path}/seoul_shp/dong/bnd_dong_11_2023_2023_2Q.shp')
dong_shp

,BASE_DATE,ADM_NM,ADM_CD,geometry
0,20230701,사직동,11010530,"POLYGON ((953553.932 1953335.741, 953555.211 1..."
1,20230701,삼청동,11010540,"POLYGON ((954025.242 1953916.389, 954026.972 1..."
2,20230701,부암동,11010550,"POLYGON ((952490.380 1956548.821, 952497.594 1..."
3,20230701,평창동,11010560,"POLYGON ((953683.828 1959209.871, 953665.283 1..."
4,20230701,한남동,11030740,"POLYGON ((956238.296 1950166.610, 956237.942 1..."
...,...,...,...,...
421,20230701,가회동,11010600,"POLYGON ((954894.795 1954614.580, 954888.290 1..."
422,20230701,종로1·2·3·4가동,11010610,"POLYGON ((954918.389 1954371.538, 954926.411 1..."
423,20230701,방배2동,11220630,"POLYGON ((954967.724 1942969.970, 954985.115 1..."
424,20230701,미아동,11090700,"POLYGON ((958301.054 1959603.612, 958292.609 1..."


In [58]:
dong_shp['ADM_CD'] =  (dong_shp['ADM_CD']).astype(int)
# hang['ADM_CD'] =  hang['ADM_CD'].astype(int)

temp2 = dong_shp.merge(temp,
               how = 'left',
               on = 'ADM_CD')

In [59]:
temp2

,BASE_DATE,ADM_NM,ADM_CD,geometry,시군구,행정동1,법정동1,행정동코드,법정동코드,행정동2,법정동2
0,20230701,사직동,11010530,"POLYGON ((953553.932 1953335.741, 953555.211 1...",종로구,사직동,통의동,1111053000,1111010600,사직동,통의동
1,20230701,사직동,11010530,"POLYGON ((953553.932 1953335.741, 953555.211 1...",종로구,사직동,적선동,1111053000,1111010700,사직동,적선동
2,20230701,사직동,11010530,"POLYGON ((953553.932 1953335.741, 953555.211 1...",종로구,사직동,체부동,1111053000,1111011200,사직동,체부동
3,20230701,사직동,11010530,"POLYGON ((953553.932 1953335.741, 953555.211 1...",종로구,사직동,필운동,1111053000,1111011300,사직동,필운동
4,20230701,사직동,11010530,"POLYGON ((953553.932 1953335.741, 953555.211 1...",종로구,사직동,내자동,1111053000,1111011400,사직동,내자동
...,...,...,...,...,...,...,...,...,...,...,...
736,20230701,종로1·2·3·4가동,11010610,"POLYGON ((954918.389 1954371.538, 954926.411 1...",종로구,종로1.2.3.4가동,원남동,1111061500,1111015900,종로1.2.3.4가동,원남동
737,20230701,종로1·2·3·4가동,11010610,"POLYGON ((954918.389 1954371.538, 954926.411 1...",종로구,종로1.2.3.4가동,종로4가,1111061500,1111016100,종로1.2.3.4가동,종로4가
738,20230701,방배2동,11220630,"POLYGON ((954967.724 1942969.970, 954985.115 1...",서초구,방배2동,방배동,1165061000,1165010100,방배2동,방배동
739,20230701,미아동,11090700,"POLYGON ((958301.054 1959603.612, 958292.609 1...",강북구,미아동,미아동,1130553500,1130510100,미아동,미아동


In [60]:
temp2[temp2.isna().any(axis = 1)]

,BASE_DATE,ADM_NM,ADM_CD,geometry,시군구,행정동1,법정동1,행정동코드,법정동코드,행정동2,법정동2
